In [1]:
import pdfplumber as pp
import pandas as pd

In [2]:
pdf = pp.open("pdf0116.pdf")

# pdf의 필요한 테이블을 데이터프레임으로 저장

In [4]:
pages = [i for i in range(19,32+1)]
pages = pages + [36, 40, 44, 48, 54, 55, 60,
                64, 69, 74, 79, 80, 86, 87, 
                91, 94, 97, 101, 106, 110, 114,
                120, 121]
print(len(pages), pages)

37 [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 36, 40, 44, 48, 54, 55, 60, 64, 69, 74, 79, 80, 86, 87, 91, 94, 97, 101, 106, 110, 114, 120, 121]


In [36]:
turn = [[1,2], [1,2], [1], [1,2,3], [1,2], [1,2,3],[1,2],
       [1,2],[1,2],[1,2,3,4],[1,2],[1,2],[1,2,3],[1,2,3],
       [1],[1],[1],[1],[1],[1], [1],[1],[1],[1],[1],
       [1],[1],[1],[1],[2],[2], [1],[1],[1],[1],[2],[1]]

# import numpy as np
# test = np.array(turn) -> 왜 안될까
# test.size
cnt = 0
for i in range(len(turn)):
    for j in range(len(turn[i])):
        cnt+=1

print(len(turn), cnt, turn)

37 56 [[1, 2], [1, 2], [1], [1, 2, 3], [1, 2], [1, 2, 3], [1, 2], [1, 2], [1, 2], [1, 2, 3, 4], [1, 2], [1, 2], [1, 2, 3], [1, 2, 3], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [2], [2], [1], [1], [1], [1], [2], [1]]


In [29]:
tables= []
for i, p in enumerate(pages):
    # 해당 페이지에 존재하는 모든 테이블 추출
    page_tables = pdf.pages[p-1].extract_tables()
    #print(type(page_tables)) -> list
    for t in turn[i]:
        tables.append(pd.DataFrame(page_tables[t-1]))

In [30]:
len(tables)

56